In [1]:
!pip install langchain langchain-google-genai PyPDF2 faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [3]:
# Import Libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import faiss


In [5]:
# Step-2 Load and extract text from PDF
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_text = load_pdf("/content/Python_Fundamentals.ipynb - Colab.pdf")

In [7]:
# Step -3 Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(pdf_text)


In [8]:
chunks

["Python is a high-level, general-purpose programming language known for its:\nReadability: Syntax resembles plain English, making it beginner-friendly.\nVersatility: Applicable to various domains like web development, data science, machine\nlearning, and automation.\nLarge Standard Library: Rich collection of built-in modules and functions for diverse tasks.\nOpen-Source Community: Extensive support and contributions from a vast developer\ncommunity.\nKey Features of Python:\nInterpreted Language: Code is executed line by line without prior compilation, offering\nfaster development cycles.\nDynamic Typing: No need to explicitly declare variable data types. Python infers types\nbased on assigned values.\nObject-Oriented Programming (OOP): Supports object-oriented concepts like classes,\nobjects, inheritance, and polymorphism for code organization and reusability.\nStrong Libraries and Frameworks: Abundant third-party libraries and frameworks extend\nPython's capabilities for specific d

In [9]:
# Step-4 Generate embeddings with Google Gemini
GOOGLE_API_KEY = "<API_KEY>" # your API Key
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
chunk_embeddings = [embeddings.embed_query(chunk) for chunk in chunks]

In [10]:
chunk_embeddings

[[0.042581599205732346,
  -0.0591748021543026,
  -0.007640151306986809,
  -0.001290480955503881,
  0.030749455094337463,
  0.0009120950708165765,
  0.04365837946534157,
  0.005570749286562204,
  0.06584455072879791,
  0.016667833551764488,
  0.018488993868231773,
  0.014749819412827492,
  -0.0022117928601801395,
  -0.01792897656559944,
  -0.009517164900898933,
  -0.007395649794489145,
  0.021190598607063293,
  0.0016653507482260466,
  -0.006009071134030819,
  0.006454467307776213,
  -0.01567554846405983,
  -0.058396000415086746,
  0.04769955947995186,
  -0.07035665214061737,
  -0.011692031286656857,
  -0.03297374024987221,
  -0.01625700853765011,
  0.009389379993081093,
  0.0071972026489675045,
  0.056743014603853226,
  -0.012167781591415405,
  0.01859094202518463,
  -0.046224240213632584,
  0.0001124151167459786,
  -0.04462902992963791,
  -0.005502534564584494,
  -0.03357425332069397,
  0.00732573214918375,
  0.010121235623955727,
  0.02444617822766304,
  0.023949313908815384,
  0.008

In [11]:
# Faiss
import numpy as np

def build_faiss_index(embeddings):
    embeddings = np.array(embeddings).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index


In [12]:
# Step 1: Embed the question
question = "What is functions in python with source code?"
question_embedding = [embeddings.embed_query(question)]

def rag_query(question, index, text_chunks):
    # Step 1: Embed the question
    question_embedding = embeddings.embed_query(question)
    # Convert to numpy array and reshape for Faiss
    question_embedding = np.array([question_embedding]).astype("float32")

    # Step 2: Retrieve relevant chunks
    distances, indices = index.search(question_embedding, k=5)
    relevant_text = " ".join([text_chunks[idx] for idx in indices[0]])
    return relevant_text


In [13]:
index = build_faiss_index(chunk_embeddings)
data_extract = rag_query(question,index,chunks)

In [14]:
formatted_text = data_extract.replace("\n \n", "\n")
formatted_text

'https://colab.research.google.com/drive/1u5zBzLZmTve9h7oB0IimyHnxPyeKTeYd#printMode=true 11/30# Fuction calling\nprint(greet("Nick"))\nHello, Nick! Welcome abroad.!\nNone\n# Function Example in Python\ndef calculate_area (length, width):\n  """Calculates and returns the area of a rectangle. """\n  return length * width\narea = calculate_area (5, 3)\nprint(f"Area of rectangle: {area}")  # Output: Area of rectangle: 15\nArea of rectangle: 15\ndef greet(name):\n  """Prints a greeting message."""\n  print(f"Hello, {name}!")\ngreet("Yaswanth" )  # Pass "Yaswanth" as an argument\ngreet("Nick")    # Pass "Nick" as an argument\ngreet("Cherry" )    # Pass "Cherry" as an argument\nHello, Yaswanth!\nHello, Nick!\nHello, Cherry!\n# Built in Functions\ndir(__builtins__ )\n\ueddb [\'ArithmeticError\',\n \'AssertionError\',\n \'AttributeError\',\n \'BaseException\',\n \'BaseExceptionGroup\',\n \'BlockingIOError\',\n \'BrokenPipeError\',\n \'BufferError\',\n \'BytesWarning\',\n \'ChildProcessError\',

In [15]:
# Using Sentence Transformer Embeddings

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_transformers_embeddings = model.encode(chunks, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# Convert embeddings to numpy array
embeddings_np = sentence_transformers_embeddings.cpu().numpy()

# Initialize FAISS index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings_np)

In [18]:
def search(query, top_k=1):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(query_embedding, top_k)
    results = [chunks[idx] for idx in indices[0]]
    return results

query = "Explain for loop with example in this Python?"
relevant_chunks = search(query)

for i, chunk in enumerate(relevant_chunks, 1):
    formatted_text = chunk.replace("\n \n", "\n")

    print(f"Result {formatted_text}")


Result #3. statements
#4. Updation
# While loop
counter = 0
while counter <= 5:
  print("Counter : " ,counter)
  counter += 1
Counter :  0
Counter :  1
Counter :  2
Counter :  3
Counter :  4
Counter :  5
Loops provide a powerful mechanism for repetitive tasks, but sometimes you need more
control over their execution.
Python offers the break and continue statements to alter the default behavior of loopsPython controlling statements in loops: break and
continue5/12/25, 8:45 PM Python_Fundamentals.ipynb - Colab
https://colab.research.google.com/drive/1u5zBzLZmTve9h7oB0IimyHnxPyeKTeYd#printMode=true 9/301. break statement:
The break statement terminates the loop it's within, immediately exiting the loop's
execution.
Control flow jumps to the code following the loop.
2. continue statement:
The continue statement skips the remaining code block of the current iteration in a loop.
The loop control jumps back to the beginning of the next iteration.
Note :
